In [1]:
device = 'mps'


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import  tqdm.notebook as tqdm
from ema_pytorch import EMA


In [3]:
batch_size = 128

transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((32, 32)),
])

train_dataset = torchvision.datasets.MNIST('data', train=True, download=True, transform=transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, persistent_workers=True)

print(f"Training with {len(train_dataset)} samples")


Training with 60000 samples


In [4]:
class Generator(nn.Module):
    def __init__(self, latent_dim=128, output_dim=1, num_classes=10):
        """ "Conditional DCGAN Generator"""
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes
        self.output_dim = output_dim

        self.embed = nn.Embedding(num_classes, latent_dim)

        self.fc = nn.Linear(latent_dim, 4 * 4 * 256)
        self.bn0 = nn.BatchNorm2d(256)

        self.deconv1 = nn.ConvTranspose2d(256, 128, 4, 2, 1)
        self.bn1 = nn.BatchNorm2d(128)

        self.deconv2 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(64)

        self.deconv3 = nn.ConvTranspose2d(64, output_dim, 4, 2, 1)

    def forward(self, z, y):
        y = self.embed(y)
        x = self.fc(z)
        x = x.view(-1, 256, 4, 4)
        x = F.relu(self.bn0(x))

        x = F.relu(self.bn1(self.deconv1(x)))
        x = F.relu(self.bn2(self.deconv2(x)))
        x = torch.tanh(self.deconv3(x))
        return x


class Discriminator(nn.Module):
    def __init__(self, input_dim=1, num_classes=10):
        super(Discriminator, self).__init__()
        self.num_classes = num_classes
        self.input_dim = input_dim

        self.embed = nn.Embedding(num_classes, 4 * 4)

        self.conv1 = nn.Conv2d(input_dim, 64, 4, 2, 1)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(64, 128, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.conv3 = nn.Conv2d(128, 256, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(256)

        self.fc = nn.Linear(4 * 4 * 257, 1)

    def forward(self, x, y):
        y = self.embed(y).view(-1, 1, 4, 4)
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2)
        x = torch.cat([x, y], dim=1).flatten(1)
        x = self.fc(x)
        return x

    def grad_penalty(self, real: torch.Tensor, fake: torch.Tensor, y: torch.Tensor):
        alpha = torch.rand(real.size(0), 1, 1, 1).to(real.device)
        interpolates = (alpha * real + (1 - alpha) * fake).requires_grad_(True)
        d_interpolates = self(interpolates, y)
        gradients = torch.autograd.grad(
            outputs=d_interpolates,
            inputs=interpolates,
            grad_outputs=torch.ones(d_interpolates.size()).to(real.device),
            create_graph=True,
            retain_graph=True,
            only_inputs=True,
        )[0]
        gradients = gradients.view(gradients.size(0), -1)
        gp = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
        return gp


generator = Generator().to(device)
discriminator = Discriminator().to(device)


In [5]:
print(f"Generator has {sum(p.numel() for p in generator.parameters() if p.requires_grad):,} parameters")
print(f"Discriminator has {sum(p.numel() for p in discriminator.parameters() if p.requires_grad):,} parameters")


Generator has 1,187,137 parameters
Discriminator has 662,001 parameters


In [6]:
optim_g = optim.AdamW(generator.parameters(), lr=2e-4, betas=(0.5, 0.999), weight_decay=4e-5)
optim_d = optim.AdamW(discriminator.parameters(), lr=2e-4, betas=(0.5, 0.999), weight_decay=4e-5)

sched_g = optim.lr_scheduler.CosineAnnealingLR(optim_g, 5_000)
sched_d = optim.lr_scheduler.CosineAnnealingLR(optim_d, 5_000)

ema_d = EMA(discriminator, beta=0.999, update_after_step=1000, update_every=100)
ema_g = EMA(generator, beta=0.999, update_after_step=1000, update_every=100)


In [7]:
generator.train()
discriminator.train()

max_iters = 10_000

pbar = tqdm.tqdm(range(0, max_iters, batch_size), total=max_iters//batch_size, desc="Training", unit="batch")

for i in pbar:
    for batch in train_loader:
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        
        # Train discriminator
        z = torch.randn(x.size(0), 128, device=device)
        fake = generator(z, y)
        real_score = discriminator(x, y)
        fake_score = discriminator(fake, y)
        
        loss_d = -real_score.mean() + fake_score.mean() + 10 * discriminator.grad_penalty(x, fake, y) + 0.001 * (real_score ** 2).mean()
        
        optim_d.zero_grad()
        loss_d.backward()
        optim_d.step()

        # Train generator
        noise = torch.randn(x.size(0), 128, device=device)
        fake = generator(noise, y)
        fake_score = discriminator(fake, y)
        loss_g = -fake_score.mean()
        
        optim_g.zero_grad()
        loss_g.backward()
        optim_g.step()
        
        ema_d.update()
        ema_g.update()
        
    
        pbar.set_postfix_str(f"Loss D: {loss_d.item():.2f}, Loss G: {loss_g.item():.2f}, Sliced W: {real_score.mean() - fake_score.mean():.2f}")

        sched_g.step()
        sched_d.step()


Training:   0%|          | 0/78 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [ ]:
generator.eval()
discriminator.eval()

with torch.no_grad():
    img_count = 10
    noise = torch.randn(img_count**2, 128, device=device)
    classes = torch.arange(img_count, device=device).repeat(img_count)

    fake = generator(noise, classes)
    
    grid = torchvision.utils.make_grid(fake, nrow=img_count).cpu()
    
    plt.figure(figsize=(20, 20))
    plt.imshow(grid.permute(1, 2, 0))
    plt.axis('off')
    plt.show()


In [ ]:
with torch.no_grad():
    noise = torch.randn(64, 100, device=device)
    fake = generator(noise)
    
    grid = torchvision.utils.make_grid(fake, nrow=8)
    
    plt.imshow(grid.permute(1, 2, 0))
    plt.axis('off')
    plt.show()


In [ ]:
# # make a new folder in the current directory called "generated" to save the images
# import os
# import time

# if not os.path.exists("generated"):
#     os.makedirs("generated")
    

# c_time = time.strftime("%Y/%m/%d-%H:%M:%S")

# if not os.path.exists(f"generated/{c_time}"):
#     os.makedirs(f"generated/{c_time}")


# # generate 100 images and save them in the "generated" folder
# for i in range(100):
#     noise = torch.randn(1, 100, device=device)
#     img = generator(noise).squeeze().detach().cpu().numpy()
#     plt.imsave(f"generated/{c_time}/{i}.png", img, cmap='gray')
